<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### Removing Tags 

In [1]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python
!pip install compound-word-splitter
!pip install pyenchant
!sudo apt-get install enchant
!sudo apt-get install hunspell-it

     |████████████████████████████████| 112kB 5.5MB/s 
     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 184kB 20.5MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=13a871005ecf4347dfb85be5beaffe88ad1028bd784b217c1461c851f8fbdcde
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=d7e7f9fb207dab5c628a52902d9f4f0064c52ad90681270b319d0b7a8d2f895e
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy
     |████████████████████████████████| 542kB 5.6MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp37-none-any.whl size=541554 sha256=b12ecccb303efe6719feed22186755ea6b6e7280b7f6dc3d847f67893e7c8a5c
  Stored in direct

In [2]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import splitter
import enchant
from itertools import groupby
import string

import sys
import os


In [174]:
# directory name:
input_dir = '/content/drive/My Drive/HLT/dataset_test_evalita/'
output_dir = '/content/drive/My Drive/HLT/dataset_test_evalita_preprocessed/'
input_dir_preprocessing = '/content/drive/My Drive/HLT/preprocessing/'

# Spec
pd.set_option("display.max_colwidth", None)

In [69]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [269]:
#tsv_tweets_file = open(input_dir+"haspeede2_reference_taskAB-tweets.tsv")
tsv_news_file = open(input_dir + "haspeede2_reference_taskAB-news.tsv")
#tsv_file = open(input_dir+"haspeede2_dev_taskAB.tsv")

dataset_raw = pd.read_csv(tsv_news_file,sep='\t', header=None)
#dataset_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw.rename(columns={0: "id"}, inplace=True)
dataset_raw.rename(columns={1: "text"}, inplace=True)
dataset_raw.rename(columns={2: "hs"}, inplace=True)
dataset_raw.rename(columns={3: "stereotype"}, inplace=True)

#Bad Words
f2 = open(input_dir_preprocessing+"bad_words.csv", 'r', encoding='utf8')

bad_words_set = [] #list of lowercase words

for x in f2:
    y = x.rstrip()
    y = y.lower()
    if y != '':
        bad_words_set.append(y)

In [270]:
dataset_raw

,id,text,hs,stereotype
0,11976,"Andate pure là, tanto quei fessi degli italiani.... Capito perché ci invadono? Il clandestino confessa",1,0
1,12142,Che fine spero che faccia il killer nigeriano di Pamela. La furia cieca della Meloni: le sue parole più dure,1,1
2,12088,Così i profughi ci svuotano i negozi a Pordenone,1,1
3,12030,"Così umiliano gli italiani e coccolano i clandestini. La follia del governo: ""Per loro gratis..."", e ci invadono",1,1
4,11775,"Danno soldi ai clandestini, ma ai disabili invece.... Bracconeri, il figlio autistico e il durissimo sfogo a ""Libero""",1,1
...,...,...,...,...
495,10085,"Sea Watch, il pm fa sbarcare i migranti. Salvini: denuncio chi ha aperto i porti",0,0
496,10044,Il pm fa sbarcare i migranti Il capo leghista smentito in tv,0,0
497,10602,"Pisa, il poster di Salvini con i migranti fatto dagli studenti, Ceccardi: «Va rimosso»",0,0
498,10193,Sea Watch e lo sbarco del migrante con una sola gamba: ''Che possa realizzare i suoi sogni'',0,0


# Preprocessing phase
Following the "Preprocessing" section of https://books.openedition.org/aaccademia/4832?lang=it#tocfrom1n4 we will perform:
*   List item
*   List item



### Extraction of the first feature: length of the comment

In [271]:
def comment_length(text):
    return len(text)

In [272]:
dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)

Extraction of the second feature: percentage of words written in CAPS-LOCK inside the comment.

In [273]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [274]:
dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)

 ### Removing Tags 

In [275]:
def clean_tag(text):
    return re.sub(
        r'@user', ' ', text)

In [276]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_tag)

### Replace the characters ‘&’, ‘@’ respectively in the letters ‘e’, ‘a’

In [277]:
def replace_e_a(text):
    text = re.sub(r'&', 'e', text)
    return re.sub(r'@', 'a', text)

In [278]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_e_a)

### Conversion of disguised bad words

In [279]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' ca[.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

In [280]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)

### Hashtag splitting and saving

In [281]:
def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

In [282]:
dataset_raw['hashtags'] = dataset_raw['text'].apply(find_hashtags)

In [283]:
def split_hashtags(text):
    
    text = ' ' + text + ' '
    result = re.findall(r'#\S+', text)
    
    for word in result:
        new_word = " ".join(splitter.split(word[1:].lower(), 'it_IT'))
        if len(new_word)==0:
            new_word =  word[1:]
                  
        text = text.replace(word, new_word)
        
    return text

In [284]:
dataset_raw['text'] = dataset_raw['text'].apply(split_hashtags)

 ### Removing URL 

In [285]:
def clean_URL(text):
    return re.sub(r'URL', ' ', text)

In [286]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_URL)

### Extraction of the fourth feature: number of ‘?’ or ‘!’ inside the comment.


In [287]:
def esclamations_and_questions(text):
    return text.count('!') + text.count('?')

In [288]:
dataset_raw['#?!'] = dataset_raw['text'].apply(esclamations_and_questions)

### Extraction of the fifth feature: number of ‘.’ or ‘,’ inside the comment.


In [289]:
def esclamations_and_questions(text):
    return text.count(',') + text.count('.')

In [290]:
dataset_raw['#.,'] = dataset_raw['text'].apply(esclamations_and_questions)

### Punctuation removal

In [291]:
def strip_punctuation(text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(' ', text)

In [292]:
dataset_raw['text'] = dataset_raw['text'].apply(strip_punctuation)

### Removal of nearby equal vowels, removal of nearby equal consonants if they are more than 2

In [293]:
correct_words_vowels = ['sciiti,',
 'welcomerefugees',
 'livetweet',
 'desiree',
 'canaan',
 'tweet.',
 'weekend',
 'romafeyenoord',
 'ebree,',
 'greencard',
 'creerà',
 'cooperative.',
 'moschee,',
 'cooperanti',
 'streetart',
 'khalidmasood',
 'tweet',
 'woolfe',
 'cooperazione',
 'coop',
 'seehofer',
 'speech',
 'coffee',
 'scooter',
 'street',
 'veemenza',
 'moschee.',
 'maalox.',
 'book',
 'tweet',
 'facebook:',
 'sociial,',
 'coop,',
 'canaaniti.',
 'europee,',
 'cooperative',
 'google',
 'creeranno',
 'mediterranee',
 'cooperazione',
 'cooperativa',
 '“boom”',
 'refugees',
 'moonlight',
 'imaam',
 'shooting',
 'sciiti',
 'sunniti',
 'book',
 'atee.',
 'looking',
 'week',
 'ayaan',
 'temporanee.',
 'idee.',
 'sibiliini',
 'food',
 'refugees',
 'retweeted',
 'boom',
 'keep',
 'vodoo',
 'hooligans',
 'ebree',
 'refugees',
 'speed',
 'bloomberg',
 'riina',
 'hatespeech',
 'google',
 'masood',
 'linee.',
 'boom']

In [294]:
def delete_duplicate_vowels_and_redundant_consonant (text):
    parole = text.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]    
        vocali = ['a','e','i','o','u','y']
        
        for idx,val in enumerate(a):
            if idx == 0:
                stringa += a[idx][0] 
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola.lower() not in correct_words_vowels):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
        
    return(stringa)  

In [295]:
dataset_raw['text'] = dataset_raw['text'].apply(delete_duplicate_vowels_and_redundant_consonant)

### Translation of emoticons

In [296]:
def translate_emoticon(text):
    text_result = emoji.demojize(text, language='it')
    text_result=re.sub(r':', ' ', text_result)
    return text_result

In [297]:
dataset_raw['text'] = dataset_raw['text'].apply(translate_emoticon)

In [298]:
emoticons_text = {
    '<kiss>': 'bacio',
    '<happy>': 'felice',
    '<laugh>': 'risata',
    '<sad>': 'triste',
    '<surprise>': 'sorpreso',
    '<wink>': 'occhiolino',
    '<tong>': 'faccia con lingua',
    '<annoyed>': 'annoiato',
    '<seallips>': 'labbra sigillate',
    '<angel>': 'angelo',
    '<devil>': 'diavolo',
    '<highfive>' : 'batti il cinque',
    '<heart>': 'cuore',
    '<user>' : 'persona',
}

In [299]:
def clean_emoticon_text(text):
    text_words = text.split()
    new_words  = [emoticons_text.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

In [300]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_emoticon_text)

### Replacement of the abbreviations with the respective words

In [301]:
abbr_word = {'cmq':'comunque', 'gov':'governatori', 'fb':'facebook', 'tw':'twitter', 'juve':'juventus', 'ing':'ingegnere', 
             'sx':'sinistra', 'qdo':'quando', 'rep':'repubblica', 'grz':'grazie', 'ita':'italia', 'mln':'milioni', 
             'mld':'miliardi', 'pke':'perche', 'anke':'anche', 'cm':'come', 'dlla':'della', 'dlle':'delle', 'qst':'questa',
             'ke':'che', 'nn':'non', 'sn':'sono', 'cn':'con', 'xk':'perche', 'xke':'perche', 'art':'articolo',
             'tv':'televisore', '€':'euro', 'xché':'perché', 'xké':'perché', 'pkè':'perché'}

In [302]:
def replace_abbreviation(text):
    text_words = text.split()
    new_text = ""
    for token in text_words:
      new_text  += abbr_word.get(token, token) +" "
    
    return new_text.strip()

In [303]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_abbreviation)

### Removal of the laughs

In [304]:
laughs = ['ah', 'eh', 'he' 'ih', 'hi'] #non elimina ahahahah, ma solo ah
vowels = ['a', 'e', 'i', 'o', 'u']

def clean_laughs(text):
    #s = "ahahahah ho fame io, eh eh" -> " ho fame io,"
    text_words = text.split()
    new_words  = [word for word in text_words if word not in laughs]
    
    new_text = ' '.join(new_words)
    
    for i in new_words:
        for k in vowels:
            if ('h' in i) and (len(i) >= 4):
                if (len(i) - 2) <= (i.count(k) + i.count('h')):
                    new_text = new_text.replace(i, '')
    
    return new_text

In [305]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_laughs)

### Tokenization

In [306]:
def tokenization(text):
    tknzr=SocialTokenizer(lowercase=False)
    return tknzr.tokenize(text)

In [307]:
dataset_raw['tokens'] = dataset_raw['text'].apply(tokenization)

### Extraction of the sixth feature: number of bad words in the comment

In [308]:
def number_bad_words(tokens):
    n_bad_words = 0

    for word in tokens:
        if word.lower() in bad_words_set:
            n_bad_words = n_bad_words + 1
        
    return n_bad_words

In [309]:
dataset_raw['#bad_words'] = dataset_raw['tokens'].apply(number_bad_words)

### Extraction of the seventh feature: percentage of bad words.

In [310]:
def percentage_bad_words(tokens):
    n_words = 0
    n_bad_words = 0
    
    for word in tokens:
        if word != '<' and word != '>':
            n_words = n_words + 1
    
    for word in tokens:
        if word.lower() in bad_words_set:
            n_bad_words = n_bad_words + 1
        
    return ((n_bad_words*100)//n_words)

In [311]:
dataset_raw['%bad_words'] = dataset_raw['tokens'].apply(percentage_bad_words)

In [312]:
dataset_raw.head(30)

,id,text,hs,stereotype,text_length,#C-L words,hashtags,#?!,"#.,",tokens,#bad_words,%bad_words
0,11976,Andate pure là tanto quei fessi degli italiani Capito perché ci invadono Il clandestino confessa,1,0,102,0,None,1,5,"[Andate, pure, là, tanto, quei, fessi, degli, italiani, Capito, perché, ci, invadono, Il, clandestino, confessa]",0,0
1,12142,Che fine spero che faccia il killer nigeriano di Pamela La furia cieca della Meloni le sue parole più dure,1,1,108,0,None,0,1,"[Che, fine, spero, che, faccia, il, killer, nigeriano, di, Pamela, La, furia, cieca, della, Meloni, le, sue, parole, più, dure]",0,0
2,12088,Così i profughi ci svuotano i negozi a Pordenone,1,1,48,0,None,0,0,"[Così, i, profughi, ci, svuotano, i, negozi, a, Pordenone]",0,0
3,12030,Così umiliano gli italiani e coccolano i clandestini La follia del governo Per loro gratis e ci invadono,1,1,112,0,None,0,5,"[Così, umiliano, gli, italiani, e, coccolano, i, clandestini, La, follia, del, governo, Per, loro, gratis, e, ci, invadono]",0,0
4,11775,Danno soldi ai clandestini ma ai disabili invece Bracconeri il figlio autistico e il durissimo sfogo a Libero,1,1,117,0,None,0,6,"[Danno, soldi, ai, clandestini, ma, ai, disabili, invece, Bracconeri, il, figlio, autistico, e, il, durissimo, sfogo, a, Libero]",0,0
5,11630,Emergenza Minniti annulla il viaggio e torna in Italia Immigrati invasione alle porte cose mai viste prima,1,1,110,0,None,0,3,"[Emergenza, Minniti, annulla, il, viaggio, e, torna, in, Italia, Immigrati, invasione, alle, porte, cose, mai, viste, prima]",0,0
6,11695,Gli immigrati sono matti parola di scienziati Malattie mentali e risorse lo studio horror,1,1,94,0,None,0,3,"[Gli, immigrati, sono, matti, parola, di, scienziati, Malattie, mentali, e, risorse, lo, studio, horror]",0,0
7,11866,Ho chiesto ai profughi di lavorare ma preferiscono non fare nulla,1,1,65,0,None,0,0,"[Ho, chiesto, ai, profughi, di, lavorare, ma, preferiscono, non, fare, nulla]",0,0
8,12174,Immigrati bomba sociale Subito via 600mila irregolari,1,1,53,0,None,0,0,"[Immigrati, bomba, sociale, Subito, via, 6, 0, 0, mila, irregolari]",0,0
9,11663,Immigrati così la sinistra ci costringe a tenerli tutti Meloni la denuncia Il trucco giuridico per rovinare l Italia,1,1,122,0,None,0,3,"[Immigrati, così, la, sinistra, ci, costringe, a, tenerli, tutti, Meloni, la, denuncia, Il, trucco, giuridico, per, rovinare, l, Italia]",0,0


In [313]:
dataset_raw.to_csv('test_dataset_news.csv', index=False)